# Get StrataScratch Subscribers with .subscribers() Youtube API

In [137]:
# !pip install google-auth
# !pip install google-auth-oauthlib

In [216]:
import pandas as pd
import os
import pickle
import time
from os import path
from pathlib2 import Path
# import pathlib2 as pl

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import googleapiclient.errors

load credentials

In [139]:
credentials = None
# token.pickle stores the user's credentials from previously successful logins
if os.path.exists('token.pickle'):
    print('Loading Credentials From File...')
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)

# If there are no valid credentials available, then either refresh the token or log in.
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print('Refreshing Access Token...')
        credentials.refresh(Request())
    else:
        print('Fetching New Tokens...')
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secrets.json',
            scopes=[
                'https://www.googleapis.com/auth/youtube.readonly'
            ]
        )

        flow.run_local_server(port=8080, prompt='consent')#,authorization_prompt_message='')
        credentials = flow.credentials

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as f:
            print('Saving Credentials for Future Use...')
            pickle.dump(credentials, f)

Loading Credentials From File...
Refreshing Access Token...


In [140]:
#Get first response for sub/comms subscriptions that are allowed:
def get_subcomms_subscriptions_first_response(channelId):
        api_service_name = "youtube"
        api_version = "v3"

        youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=credentials)
        request = youtube.subscriptions().list(part="snippet",channelId=channelId)
        # request = youtube.subscriptions().list(part="snippet,contentDetails",pageToken='CLYHEAE', maxResults=maxResults,mySubscribers=True)
        response = request.execute()
        return response, request, youtube

In [167]:
def get_subscriber_ids(df,response):
    for sub in response['items']:
        if sub['kind'] == 'youtube#subscription': 
            
            response_kind =sub['kind']
            publishedAt=sub['snippet']['publishedAt']
            channel_title =sub['snippet']['title']
            channel_description =sub['snippet']['description']
            channelId = sub['snippet']['resourceId']['channelId']
            sub_channelId = sub['snippet']['channelId']        

            try:
                if response['nextPageToken'] != None:
                    pageToken = response['nextPageToken']
                    df = df.append({'response_kind': response_kind, 'publishedAt':publishedAt, 'channel_title':channel_title,'channel_description':channel_description,'channelId':channelId,'sub_channelId':sub_channelId,'pageToken':pageToken},ignore_index=True)    
                else:
                    df = df.append({'response_kind': response_kind, 'publishedAt':publishedAt, 'channel_title':channel_title,'channelId':channelId,'sub_channelId':sub_channelId,'pageToken':'none'},ignore_index=True)    
            except Exception as e: 
                print('get_subs_ids error:')
                print(str(e))
                #save data in pandas df
                df = df.append({'response_kind': response_kind, 'publishedAt':publishedAt, 'channel_title':channel_title,'channelId':channelId,'sub_channelId':sub_channelId,'pageToken':'none'},ignore_index=True)
    
    return df

In [168]:
def get_subs_pagination(df,request,response,youtube):
    while 1:
        try:
            request = youtube.subscriptions().list_next(previous_request=request, previous_response=response)
            response = request.execute()
            df = get_subscriber_ids(df,response)
        
        except Exception as e: 
            print('get_subs_paginations error:')
            if 'you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.' in str(e):
                print(str(e))
                print(f'stopped at {current_chan_idx}')
                df = df[df.sub_channelId != channelId] # erase anything you just loaded because it's incomplete
                print(f'erased all from last Id  - {channelId}')    
            break
    
    return df,request,response

## Main

In [169]:
# # Manual Variables
# # start=0

# cnname = 'gofig' # channel nick name

# comments_csv = 'gofig_youtube_comments_974rows.csv'
# # start  = 734  #from last pull on gofig

In [282]:
# Manual Variables
# start= 2031

cnname = 'ThePrimeagen' # channel nick name
mchannel_id = 'UC8ENHE5xdFSwx71u3fDH5Xw' #  main channel ID for the primegean
mchannel_name = 'ThePrimeagen'  # main channel name for 'ThePrimeagen'

comments_csv = 'theprime_youtube_comments_11045rows.csv'

goal = start + 10

start

2977

In [283]:
#main
exceptions_count=0
last_start =start # in case we need to start again at the last spot
dfo = pd.read_csv(comments_csv,low_memory=False)

dfo = dfo.loc[dfo.response_kind!='youtube#video']
dfo = dfo.reset_index(drop=True)

chan_ids = dfo.authorChannelId.unique() # create chan_ids from imported df for all youtube scraping:
# chan_ids = chan_ids[1:]  #this used to be necessary ,but now you've cleaned up dfo before so you don't need it

df = pd.DataFrame()
# user_count = 0
# 

# goal = start + (len(chan_ids)-start)

#########Grab everything using this function
# df, last_user,exceptions_count = main_getter(df,chan_ids=chan_ids,start=start,exceptions_count=exceptions_count)

for channelId in chan_ids[start:goal]:
    attempting = goal - start
    current_chan_idx = list(chan_ids).index(channelId)
    total_overall = len(chan_ids)
    try:        
        response ,request, youtube = get_subcomms_subscriptions_first_response(channelId)
        #turn first response to df
        df = get_subscriber_ids(df,response)
        print('df length: ' + str(len(df)))
        
        df,request,response = get_subs_pagination(df=df,response=response ,request=request, youtube=youtube)
        last_unique_scId = df.loc[df.index[-1],'sub_channelId']
        user_count = df.sub_channelId.nunique()
        last_Id_idx = list(chan_ids).index(last_unique_scId)
        
        #tracker to make sure its doing what it says
        print(f'users collected this round: {user_count} / error ct: {exceptions_count} / attempting this round: {attempting}')
        print(f'start: {start} / at: {current_chan_idx} / goal: {goal} / All Possible Commenters: {total_overall}')
    
    except Exception as e: 
                
        #tracker to make sure its doing what it says
                  
        print(f'users collected this round: {user_count} / error ct: {exceptions_count} / attempting this round: {attempting}')
        print(f'start: {start} / at: {current_chan_idx} / goal: {goal} / All Possible Commenters: {total_overall}')

        exceptions_count +=1
        print('main getter error')
        
        if 'not allowed to access the requested subscriptions' in str(e):
            print('not allowed to access the requested subscriptions')
            pass
        
        elif 'not allowed to access the requested subscriptions' not in str(e):
            if 'you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.' in str(e):
                print(str(e))
                print(f'stopped at {current_chan_idx}')
                df = df[df.sub_channelId != channelId] # erase anything you just loaded because it's incomplete
                print(f'erased all from last Id  - {channelId}')
                break
            else:      
                print(str(e))
                df = df[df.sub_channelId != channelId] # erase anything you just loaded because it's incomplete
                print(f'erased all from last Id  - {channelId}')
                break
                pass
            
                
last_user = last_Id_idx #create last user variable to use for name

###
# last user should instead be last_Id_idx

###
df.to_csv(f'competitor_{cnname}_commenters_subscricptions_{start}_{last_user}.csv',index=False) # save as csv
print(f'Saved: competitor_{cnname}_commenters_subscriptions_{start}_{last_user}.csv') 


#create path object and filename to read/save to.
this_path = Path('/Users/davidramsey/Documents/coding/youtube_scraping/strata/all_subs_combined')
filename =f'{mchannel_name}_{mchannel_id}_coms_combined.csv'

if os.path.exists(this_path/filename):
    dfall = pd.read_csv(this_path/f'{mchannel_name}_{mchannel_id}_coms_combined.csv',lineterminator='\n',low_memory=False) #load combined csv
    print('dfall found')
    dfall = pd.concat([dfall,df]) # concat to new df
else:
    dfall = df
    print('no dfall found')

dfall = dfall.drop_duplicates() #drop allo= duplicates
dfall = dfall.reset_index(drop=True)

# save newly combined and print the number of unique users
dfall.to_csv(this_path/f'{mchannel_name}_{mchannel_id}_coms_combined.csv',index=False) ;
print('total combined Uniques: '+ str(dfall.sub_channelId.nunique() )+' / '+ str(dfo.authorChannelId.nunique())+' possible uniques')

# start=last_user #set up start for next time around
last_unique_scId = df.loc[df.index[-1],'sub_channelId']
last_Id_idx = list(chan_ids).index(last_unique_scId)

users_attempted = goal-start
users_pulled = df.sub_channelId.nunique()
length = len(dfall)
print(f'started: {start}/ ended: {current_chan_idx} / out of a total of '+ str(dfo.authorChannelId.nunique()))

print(f'users attmepted: {users_attempted} / users pulled {users_pulled} / exeptions this round: {exceptions_count}/ len of dfall: {length}' )

start = last_Id_idx +1 #change the start for next time
# time.sleep(120)
df.tail(3)

users collected this round: 111 / error ct: 0 / attempting this round: 10
start: 2977 / at: 2977 / goal: 2987 / All Possible Commenters: 3981
main getter error
not allowed to access the requested subscriptions
users collected this round: 111 / error ct: 1 / attempting this round: 10
start: 2977 / at: 2978 / goal: 2987 / All Possible Commenters: 3981
main getter error
not allowed to access the requested subscriptions
users collected this round: 111 / error ct: 2 / attempting this round: 10
start: 2977 / at: 2979 / goal: 2987 / All Possible Commenters: 3981
main getter error
not allowed to access the requested subscriptions
users collected this round: 111 / error ct: 3 / attempting this round: 10
start: 2977 / at: 2980 / goal: 2987 / All Possible Commenters: 3981
main getter error
not allowed to access the requested subscriptions
df length: 5
get_subs_ids error:
'nextPageToken'
get_subs_ids error:
'nextPageToken'
get_subs_ids error:
'nextPageToken'
get_subs_ids error:
'nextPageToken'
get

,response_kind,publishedAt,channel_title,channel_description,channelId,sub_channelId,pageToken
468,youtube#subscription,2021-12-18T10:37:52.438856Z,Windows Developer,Everything you need to know to develop great a...,UCzLbHrU7U3cUDNQWWAqjceA,UCxfsdY-BgKEecx_vS781v6A,CJYBEAA
469,youtube#subscription,2021-05-07T18:31:19.454149Z,Россия23,Телеграм: https://t.me/vesti23rossia\nВконтакт...,UCzXTNApTYz6Cwqh6Naadu1A,UCxfsdY-BgKEecx_vS781v6A,CJYBEAA
470,youtube#subscription,2016-10-26T14:00:18Z,Владислав Васин,NaN,UCzZvy1qmPn6XZujCqqbBw0A,UCxfsdY-BgKEecx_vS781v6A,none


---